# Late-fusion
Cargamos los datos de nuestras redes ya entrenadas



In [ ]:
from google.colab import files
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np

# Subir los archivos necesarios
print("Sube tus modelos y archivos .npy")
uploaded = files.upload()

# Cargar los modelos preentrenados
model1 = load_model('modelo1.h5')  # Modelo tabular
model2 = load_model('modelo_resnet50_inicial.h5')  # Modelo ResNet50

# Cargar etiquetas y predicciones intermedias para fusión tardía
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')

y_pred_val_model_a = np.load('y_pred_val_model_a.npy')  # Predicciones del modelo tabular
y_pred_test_model_a = np.load('y_pred_test_model_a.npy')
y_pred_val_model_b = np.load('val_predictions_resnet_initial.npy')  # Predicciones del modelo ResNet50
y_pred_test_model_b = np.load('test_predictions_resnet_initial.npy')

# Validar y alinear dimensiones
min_val_samples = min(len(y_pred_val_model_a), len(y_pred_val_model_b))
min_test_samples = min(len(y_pred_test_model_a), len(y_pred_test_model_b))

y_pred_val_model_a = y_pred_val_model_a[:min_val_samples]
y_pred_val_model_b = y_pred_val_model_b[:min_val_samples]
y_pred_test_model_a = y_pred_test_model_a[:min_test_samples]
y_pred_test_model_b = y_pred_test_model_b[:min_test_samples]

y_val = y_val[:min_val_samples]
y_test = y_test[:min_test_samples]

# Concatenar predicciones para la fusión tardía
X_val_fused = np.concatenate((y_pred_val_model_a, y_pred_val_model_b), axis=1)
X_test_fused = np.concatenate((y_pred_test_model_a, y_pred_test_model_b), axis=1)

# Nota: X_train_fused no está definido en el código original porque parece que estás trabajando con validación cruzada.
# Usa un conjunto de entrenamiento fusionado si es necesario.

# Definir el modelo final de fusión
input_fused = Input(shape=(X_val_fused.shape[1],))  # Tamaño de las características fusionadas
x = Dense(256, activation='relu')(input_fused)
x = Dense(128, activation='relu')(x)
final_output = Dense(3, activation='softmax')(x)  # Cambia 3 por el número de clases de tu problema

final_model = Model(inputs=input_fused, outputs=final_output)

# Compilar el modelo
final_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Configurar callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_fusion_model.keras', monitor='val_loss', save_best_only=True)

# Entrenar el modelo de fusión
final_model.fit(
    X_val_fused, y_val,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_fused, y_val),
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = final_model.evaluate(X_test_fused, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Guardar el modelo final
final_model.save("final_fusion_model.h5")
print("Modelo de fusión guardado como 'final_fusion_model.h5'.")

# Descargar características y etiquetas
files.download('final_fusion_model.h5')


Sube tus modelos y archivos .npy


Saving modelo_resnet50_inicial.h5 to modelo_resnet50_inicial.h5
Saving modelo1.h5 to modelo1.h5
Saving test_features_resnet_initial.npy to test_features_resnet_initial.npy
Saving test_predictions_resnet_initial.npy to test_predictions_resnet_initial.npy
Saving train_features_resnet_initial.npy to train_features_resnet_initial.npy
Saving train_predictions_resnet_initial.npy to train_predictions_resnet_initial.npy
Saving val_features_resnet_initial.npy to val_features_resnet_initial.npy
Saving val_predictions_resnet_initial.npy to val_predictions_resnet_initial.npy
Saving X_test_scaled.npy to X_test_scaled.npy
Saving X_train_scaled.npy to X_train_scaled.npy
Saving X_val_scaled.npy to X_val_scaled.npy
Saving y_pred_test_model_a.npy to y_pred_test_model_a.npy
Saving y_pred_val_model_a.npy to y_pred_val_model_a.npy
Saving y_test.npy to y_test.npy
Saving y_test_images.npy to y_test_images.npy
Saving y_train.npy to y_train.npy
Saving y_train_images.npy to y_train_images.npy
Saving y_val.npy t

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.4583 - loss: 1.0513 - val_accuracy: 0.7404 - val_loss: 0.9147 - learning_rate: 0.0010
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7430 - loss: 0.8762 - val_accuracy: 0.7500 - val_loss: 0.7663 - learning_rate: 0.0010
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7200 - loss: 0.7550 - val_accuracy: 0.7468 - val_loss: 0.6662 - learning_rate: 0.0010
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7650 - loss: 0.6314 - val_accuracy: 0.7372 - val_loss: 0.6268 - learning_rate: 0.0010
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6982 - loss: 0.6884 - val_accuracy: 0.7436 - val_loss: 0.6190 - learning_rate: 0.0010
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7739 - loss: 0.5622 - val_accuracy: 0.7500 - val_loss: 0.6159 - learning_rate: 0.0010
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7446 - loss: 0.6390 - val_accurac

Test Accuracy: 0.70
Modelo de fusión guardado como 'final_fusion_model.h5'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Supongamos que tienes las etiquetas reales (y_test) y las predicciones del modelo
y_pred = np.argmax(final_model.predict(X_test_fused), axis=1)

# Generar el informe de clasificación
print("Informe de Clasificación:")
print(classification_report(y_test, y_pred, target_names=['Clase 0', 'Clase 1', 'Clase 2']))

# Generar la matriz de confusión
print("Matriz de Confusión:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Informe de Clasificación:
              precision    recall  f1-score   support

     Clase 0       0.75      0.76      0.76        89
     Clase 1       0.56      0.68      0.62       101
     Clase 2       0.83      0.67      0.74       123

    accuracy                           0.70       313
   macro avg       0.71      0.70      0.70       313
weighted avg       0.72      0.70      0.70       313

Matriz de Confusión:
[[68 18  3]
 [18 69 14]
 [ 5 36 82]]


# Conclusiones de los resultados con Late-Fusion
**Late-Fusion: Combinación de Modelos**

### Modelo tabular (Red densa):
- Desempeño individual: **71% de precisión general**.
- Fortalezas: Predice mejor las clases **bajo** y **alto**.
- Debilidad: Tiene dificultades para distinguir correctamente la clase **medio**.

### Modelo de imágenes (ResNet50):
- Desempeño individual: Predice correctamente solo la clase **medio**.
- Problemas: No diferencia las clases **bajo** y **alto**.

### Lo que conseguimos con Late-Fusion:
- **Mejor balance de predicciones**: La combinación de ambos modelos permite aprovechar sus fortalezas y compensar sus debilidades.
- **Enfoque combinado**: El modelo tabular ayuda a identificar los extremos (**bajo** y **alto**), mientras que el modelo de imágenes mejora la predicción de la clase intermedia (**medio**).
- **Rendimiento más robusto**: Late-Fusion da lugar a un modelo final que integra mejor la información, reduciendo los sesgos de cada modelo por separado.

En resumen, el enfoque de Late-Fusion nos permitió construir un modelo que combina lo mejor de ambos mundos, mejorando el rendimiento general en la clasificación de precios.


# Early-fusion

In [ ]:
from google.colab import files
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow as tf

# Subir los archivos necesarios
print("Sube tus modelos y archivos .npy")
uploaded = files.upload()

# Cargar características preprocesadas directamente
print("\nCargando características preprocesadas...")
try:
    features_train_model1 = np.load('X_train_scaled.npy')  # Características tabulares
    features_val_model1 = np.load('X_val_scaled.npy')
    features_test_model1 = np.load('X_test_scaled.npy')

    features_train_model2 = np.load('train_features_resnet_initial.npy')  # Características de ResNet50
    features_val_model2 = np.load('val_features_resnet_initial.npy')
    features_test_model2 = np.load('test_features_resnet_initial.npy')

    y_train = np.load('y_train.npy')
    y_val = np.load('y_val.npy')
    y_test = np.load('y_test.npy')

    print("Características cargadas con éxito.")
except Exception as e:
    print("Error al cargar las características:", e)
    raise

# Concatenar las características
print("\nConcatenando características...")
try:
    if features_train_model1.shape[0] != features_train_model2.shape[0]:
        raise ValueError("El número de muestras no coincide entre los dos modelos.")

    X_train_fused = np.concatenate((features_train_model1, features_train_model2), axis=1)
    X_val_fused = np.concatenate((features_val_model1, features_val_model2), axis=1)
    X_test_fused = np.concatenate((features_test_model1, features_test_model2), axis=1)

    print("Características concatenadas con éxito.")
except Exception as e:
    print("Error al concatenar características:", e)
    raise

# Construir el clasificador final
print("\nConstruyendo el clasificador final...")
try:
    num_classes = len(np.unique(y_train))  # Número de clases dinámico
    input_fused = Input(shape=(X_train_fused.shape[1],))
    x = Dense(256, activation='relu')(input_fused)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)
    final_output = Dense(num_classes, activation='softmax')(x)

    final_classifier = Model(inputs=input_fused, outputs=final_output)
    final_classifier.compile(optimizer=Adam(learning_rate=0.0005),
                             loss='sparse_categorical_crossentropy',
                             metrics=['accuracy'])
    print("Clasificador construido y compilado con éxito.")
except Exception as e:
    print("Error al construir el clasificador:", e)
    raise

# Configurar callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint('best_early_fusion_model.keras', monitor='val_loss', save_best_only=True)
]

# Entrenar el clasificador final
print("\nEntrenando el clasificador final...")
try:
    final_classifier.fit(
        X_train_fused, y_train,
        validation_data=(X_val_fused, y_val),
        epochs=20,
        batch_size=32,
        callbacks=callbacks
    )
    print("Entrenamiento completado.")
except Exception as e:
    print("Error durante el entrenamiento:", e)
    raise

# Evaluar el modelo en el conjunto de prueba
print("\nEvaluando el modelo...")
try:
    test_loss, test_acc = final_classifier.evaluate(X_test_fused, y_test)
    print(f"Test Accuracy: {test_acc:.2f}")
except Exception as e:
    print("Error durante la evaluación:", e)
    raise

# Guardar el modelo final
try:
    final_classifier.save("early_fusion_classifier.keras")
    print("Modelo de fusión temprana guardado como 'early_fusion_classifier.keras'.")
    files.download('early_fusion_classifier.keras')
except Exception as e:
    print("Error al guardar el modelo:", e)
    raise


Sube tus modelos y archivos .npy


Saving modelo_resnet50_inicial.h5 to modelo_resnet50_inicial.h5
Saving modelo1.h5 to modelo1.h5
Saving test_features_resnet_initial.npy to test_features_resnet_initial.npy
Saving test_predictions_resnet_initial.npy to test_predictions_resnet_initial.npy
Saving train_features_resnet_initial.npy to train_features_resnet_initial.npy
Saving train_predictions_resnet_initial.npy to train_predictions_resnet_initial.npy
Saving val_features_resnet_initial.npy to val_features_resnet_initial.npy
Saving val_predictions_resnet_initial.npy to val_predictions_resnet_initial.npy
Saving X_test_scaled.npy to X_test_scaled.npy
Saving X_train_scaled.npy to X_train_scaled.npy
Saving X_val_scaled.npy to X_val_scaled.npy
Saving y_pred_test_model_a.npy to y_pred_test_model_a.npy
Saving y_pred_val_model_a.npy to y_pred_val_model_a.npy
Saving y_test.npy to y_test.npy
Saving y_test_images.npy to y_test_images.npy
Saving y_train.npy to y_train.npy
Saving y_train_images.npy to y_train_images.npy
Saving y_val.npy t

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
import numpy as np

# Cargar el modelo de fusión temprana
try:
    early_fusion_model = load_model("early_fusion_classifier.keras")
    print("Modelo de fusión temprana cargado correctamente.")
except Exception as e:
    raise RuntimeError(f"Error al cargar el modelo de fusión temprana: {e}")

# Verificar que X_test_fused y y_test estén definidos y tengan dimensiones válidas
try:
    print(f"Forma de X_test_fused: {X_test_fused.shape}")
    print(f"Forma de y_test: {y_test.shape}")
except NameError as e:
    raise NameError("`X_test_fused` o `y_test` no están definidos. Asegúrate de haberlos cargado correctamente.") from e

# Hacer predicciones con el modelo de fusión temprana
try:
    y_pred = np.argmax(early_fusion_model.predict(X_test_fused, batch_size=32), axis=1)
    print("Predicciones realizadas correctamente.")
except Exception as e:
    raise RuntimeError(f"Error al realizar predicciones: {e}")

# Generar el informe de clasificación
try:
    print("Informe de Clasificación para Early-Fusion:")
    print(classification_report(y_test, y_pred, target_names=['Clase 0', 'Clase 1', 'Clase 2']))
except ValueError as e:
    raise ValueError(f"Error al generar el informe de clasificación. Verifica el formato de `y_test` y `y_pred`: {e}")

# Generar la matriz de confusión
try:
    print("Matriz de Confusión para Early-Fusion:")
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(conf_matrix)
except ValueError as e:
    raise ValueError(f"Error al generar la matriz de confusión: {e}")




Modelo de fusión temprana cargado correctamente.
Forma de X_test_fused: (313, 369)
Forma de y_test: (313,)
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicciones realizadas correctamente.
Informe de Clasificación para Early-Fusion:
              precision    recall  f1-score   support

     Clase 0       0.78      0.73      0.76        89
     Clase 1       0.58      0.67      0.62       101
     Clase 2       0.82      0.75      0.78       123

    accuracy                           0.72       313
   macro avg       0.73      0.72      0.72       313
weighted avg       0.73      0.72      0.72       313

Matriz de Confusión para Early-Fusion:
[[65 21  3]
 [16 68 17]
 [ 2 29 92]]


### **Conclusiones de los resultados con Early-Fusion**

#### **Desempeño general:**
- Precisión general: **72%**, comparable al enfoque de Late-Fusion.
- Buen balance entre clases, aunque con un rendimiento ligeramente inferior en la clase media (Clase 1).

#### **Análisis de clases:**
- **Clase 0 (bajo):**
  - **Precisión:** 78%
  - **Recall:** 73%
  - Buen desempeño al identificar correctamente esta clase, con pocos falsos positivos y falsos negativos.

- **Clase 1 (medio):**
  - **Precisión:** 58%
  - **Recall:** 67%
  - Presenta desafíos similares a Late-Fusion, con un recall decente, pero mayor cantidad de falsos positivos, reduciendo la precisión.

- **Clase 2 (alto):**
  - **Precisión:** 82%
  - **Recall:** 75%
  - Buen rendimiento al predecir esta clase, aunque no tan preciso como para la Clase 0.

#### **Matriz de confusión:**
- Se observa confusión entre clases adyacentes (Clase 0 con Clase 1, y Clase 1 con Clase 2).
- Mantiene un balance razonable a pesar de las confusiones.

---

### **Comparación con Late-Fusion**

#### **Desempeño general:**
- **Early-Fusion:** Precisión **72%**.
- **Late-Fusion:** Precisión **71%**, con mejor balance entre clases, compensando debilidades de los modelos individuales.

#### **Predicción de la clase media (Clase 1):**
- **Late-Fusion:** Mejora la predicción gracias a su enfoque combinado que aprovecha las fortalezas de ambos modelos.
- **Early-Fusion:** Al procesar todas las características juntas, no logra optimizar tan bien esta clase.

#### **Ventajas de Early-Fusion:**
- **Simplicidad:** Integra las características desde el principio, eliminando la necesidad de procesar salidas separadas.
- **Desempeño sólido:** Particularmente en las clases extrema baja y alta.

#### **Ventajas de Late-Fusion:**
- **Flexibilidad:** Combina las predicciones finales de los modelos, ajustando el peso de cada uno según las necesidades.
- **Mejor balance:** Es más robusto en la clase media, una debilidad de Early-Fusion.

---

### **Conclusión general:**
- **Early-Fusion:**  
  Enfoque directo y efectivo, con buen desempeño general. Sin embargo, presenta dificultades para balancear las clases, especialmente en la clase media.

- **Late-Fusion:**  
  Ofrece mejor balance al aprovechar las fortalezas específicas de los modelos tabular e imagen. Es ideal cuando el equilibrio entre las clases es prioritario.

**Resumen:**  
- Para maximizar el balance entre clases, **Late-Fusion** es preferido.  
- **Early-Fusion** es una alternativa menos compleja, con resultados sólidos en las clases extrema baja y alta.
